In [84]:
# Imports
from __future__ import absolute_import
from __future__ import division # Use a//b to do integer division
from __future__ import print_function # Makes print a function, use print("Hello World")

import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf

In [85]:
print("yo")

yo


In [86]:
# Get labeled data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [123]:
# Data exploration
# Use mnist.train.next_batch to get a batch of data
x_explore, y_explore = mnist.train.next_batch(2)
print(type(x_explore))
print(x_explore.shape)
print(x_explore)
print(type(y_explore))
print(y_explore.shape)
print(y_explore)

n_features = x_explore.shape[1]
n_output = y_explore.shape[1]

x_explore /= np.max(x_explore)

<type 'numpy.ndarray'>
(2, 784)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
<type 'numpy.ndarray'>
(2, 10)
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


# Data type
Images are 28x28 greyscale (MNIST description on Yann LeCunn's website)

In [124]:
# Initialize DNN architecture
# Input : layers =  [n_features, n1, n2, ... , n_output]

def initialize_DNN(layers):
    parameters = {}
    for i in range(1, len(layers)):
        parameters["W" + str(i)] = np.random.randn(layers[i], layers[i-1]) * (1.0 / layers[i-1])
        parameters["b" + str(i)] = np.zeros((layers[i], 1))
        
    return parameters

In [125]:
test_param = initialize_DNN([4,3,2])
print(test_param)

{'b2': array([[ 0.],
       [ 0.]]), 'b1': array([[ 0.],
       [ 0.],
       [ 0.]]), 'W1': array([[ 0.45813492,  0.50746729,  0.04959326,  0.49361363],
       [ 0.01219896,  0.72663458, -0.2120326 , -0.09057736],
       [ 0.3024943 , -0.23985896,  0.50528011,  0.17214653]]), 'W2': array([[-0.28361862, -0.05729338, -0.25124414],
       [-0.01120927,  0.27350801,  0.11972458]])}


In [126]:
def apply_activation(Z, activation):
    
    if (activation == "tanh"):
        return np.tanh(Z)
    elif (activation == "sigmoid"):
        return 1.0 / (1.0 + np.exp(-Z))
    
    #if activation == "relu":
    #    return tf.nn.relu(Z)
    #    #return Z * (Z > 0)

In [127]:
# Forward propagation

def forward_propagation(X, layers, parameters, activation):
    A_prev = X.T
    cache = {}
    n_layers = len(layers)
    
    for i in range(1, n_layers-1):
        W = parameters["W" + str(i)]
        b = parameters["b" + str(i)]
        Z = np.dot(W,A_prev) + b 
        A = apply_activation(Z, activation = "tanh")
        
        cache["Z" + str(i)] = Z
        cache["A" + str(i)] = A
        
        A_prev = A
    
    # Output layer
    W = parameters["W" + str(n_layers-1)]
    b = parameters["b" + str(n_layers-1)]
    Z = np.dot(W,A_prev) + b 
    A = apply_activation(Z, activation = "sigmoid") # This should be a softmax
        
    cache["Z" + str(n_layers-1)] = Z
    cache["A" + str(n_layers-1)] = A    
        
    return parameters, cache

In [128]:
parameters = initialize_DNN(layers)

_, cache = forward_propagation(x_explore, layers, parameters, activation="relu")
print(cache)

{'A1': array([[-0.00285962,  0.00337693],
       [-0.00728551,  0.02315124],
       [-0.01494057,  0.00721465],
       [-0.02039987,  0.00262549],
       [-0.0147092 , -0.00639248]]), 'A2': array([[ 0.50008659,  0.49830257],
       [ 0.5007718 ,  0.49948983],
       [ 0.50053388,  0.49888125],
       [ 0.50090101,  0.50102691],
       [ 0.49846668,  0.50348128],
       [ 0.49967482,  0.49809519],
       [ 0.50233387,  0.50018957],
       [ 0.50001385,  0.49961647],
       [ 0.50080949,  0.49945294],
       [ 0.50072151,  0.50190094]]), 'Z1': array([[-0.00285963,  0.00337694],
       [-0.00728564,  0.02315537],
       [-0.01494168,  0.00721478],
       [-0.0204027 ,  0.0026255 ],
       [-0.01471026, -0.00639257]]), 'Z2': array([[  3.46370422e-04,  -6.78974384e-03],
       [  3.08719368e-03,  -2.04066119e-03],
       [  2.13553163e-03,  -4.47499638e-03],
       [  3.60403306e-03,   4.10763561e-03],
       [ -6.13329193e-03,   1.39253612e-02],
       [ -1.30072608e-03,  -7.61926581e-03],

In [129]:
# Cost funtion

def compute_cost(Y, Y_hat):
    m = Y.shape[1]
    
    cost = - (1.0 / m) * np.sum((np.dot(Y.T, np.log(Y_hat)) + np.dot((1.0-Y).T, np.log((1.0-Y_hat)))))
    return cost

In [130]:
cost = compute_cost(y_explore.T, cache["A" + str(len(layers)-1)])
print(cost)

13.8801984796


In [131]:
# Back Prop:

def backward_propagation(Y, cache, parameters, layers):
    m = Y.shape[1]
    n_layers = len(layers)
    grads = {}

    # Derivative of a softmax??
    grads["dZ" + str(n_layers-1)] = Y - cache("A" + str(n_layers-1))
    
        
        
    

IndentationError: expected an indented block (<ipython-input-131-6a2273b56cc5>, line 13)